#Купил значит я себе рахат-лукум, как в Греции

In [7]:
!pip install catboost # Теперь я вич инфецированный
!pip install unrar

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.4 MB/s eta 0:00:00


In [8]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [9]:
!unzip /content/drive/MyDrive/Попытки-в-Catboost/new-york-city-taxi-fare-prediction.zip

unzip:  cannot find or open /content/drive/MyDrive/Попытки-в-Catboost/new-york-city-taxi-fare-prediction.zip, /content/drive/MyDrive/Попытки-в-Catboost/new-york-city-taxi-fare-prediction.zip.zip or /content/drive/MyDrive/Попытки-в-Catboost/new-york-city-taxi-fare-prediction.zip.ZIP.


In [2]:
# Функция для расчета расстояния между координатами
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Радиус Земли в километрах
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
    return R * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))

In [3]:
# Загрузка данных
data = pd.read_csv('/content/train.csv', nrows=1000000)
data = data.dropna() # Убираем пустые значения

NameError: name 'pd' is not defined

In [ ]:
# Преобразование даты и времени
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'], errors='coerce')
data = data.dropna(subset=['pickup_datetime'])

In [ ]:
# Создание временных признаков
data['year'] = data['pickup_datetime'].dt.year
data['month'] = data['pickup_datetime'].dt.month
data['day'] = data['pickup_datetime'].dt.day
data['day_of_week'] = data['pickup_datetime'].dt.dayofweek
data['hour'] = data['pickup_datetime'].dt.hour

In [ ]:
# Создание признака расстояния
data['distance'] = haversine_distance(data['pickup_latitude'], data['pickup_longitude'],
                                      data['dropoff_latitude'], data['dropoff_longitude'])

In [ ]:
features = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
            'passenger_count', 'year', 'month', 'day', 'day_of_week', 'hour', 'distance']
target = 'fare_amount'

X = data[features]
y = data[target]

# Разделение данных
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Создание и обучение модели
model = CatBoostRegressor(iterations=1100, learning_rate=0.1, depth=10, loss_function='RMSE', verbose=100)
model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=100)

0:	learn: 9.0237071	test: 9.2385454	best: 9.2385454 (0)	total: 85.5ms	remaining: 1m 25s
100:	learn: 3.6716586	test: 4.4990351	best: 4.4990351 (100)	total: 8.96s	remaining: 1m 19s
200:	learn: 3.2614361	test: 4.4243813	best: 4.4241794 (199)	total: 17.8s	remaining: 1m 10s
300:	learn: 2.9943356	test: 4.4140216	best: 4.4140216 (300)	total: 23.2s	remaining: 53.8s
400:	learn: 2.8040104	test: 4.4087989	best: 4.4066015 (341)	total: 26.9s	remaining: 40.2s
500:	learn: 2.6464837	test: 4.4176877	best: 4.4066015 (341)	total: 30.6s	remaining: 30.5s
600:	learn: 2.5198596	test: 4.4197194	best: 4.4066015 (341)	total: 38.4s	remaining: 25.5s
700:	learn: 2.4086625	test: 4.4233116	best: 4.4066015 (341)	total: 47.1s	remaining: 20.1s
800:	learn: 2.3094694	test: 4.4236104	best: 4.4066015 (341)	total: 56.8s	remaining: 14.1s
900:	learn: 2.2173621	test: 4.4270161	best: 4.4066015 (341)	total: 1m 2s	remaining: 6.86s
999:	learn: 2.1370928	test: 4.4317680	best: 4.4066015 (341)	total: 1m 6s	remaining: 0us

bestTest = 

In [ ]:
# Оценка модели
y_pred = model.predict(X_val)
rmse = math.sqrt(mean_squared_error(y_val, y_pred))
print(f"RMSE на тестовой выборке: {rmse:.2f}")

RMSE на тестовой выборке: 4.41


In [ ]:
# Загрузка тестовых данных
test_data = pd.read_csv('/content/test.csv')

# Предобработка тестовых данных
test_data['pickup_datetime'] = pd.to_datetime(test_data['pickup_datetime'], errors='coerce')
test_data['year'] = test_data['pickup_datetime'].dt.year
test_data['month'] = test_data['pickup_datetime'].dt.month
test_data['day'] = test_data['pickup_datetime'].dt.day
test_data['day_of_week'] = test_data['pickup_datetime'].dt.dayofweek
test_data['hour'] = test_data['pickup_datetime'].dt.hour

test_data['distance'] = haversine_distance(test_data['pickup_latitude'], test_data['pickup_longitude'],
                                      test_data['dropoff_latitude'], test_data['dropoff_longitude'])

In [ ]:
predictions = model.predict(test_data[features]) # ТО САМОЕ ПРЕДСКАЗАНИЕ

# Создаем DataFrame для отправки
submission = pd.DataFrame({
    'key': test_data['key'],
    'fare_amount': predictions
    })

# Сохранение результатов
submission.to_csv('taxi_fare_submission.csv', index=False)
print("Файл taxi_fare_submission.csv успешно создан.")

Файл taxi_fare_submission.csv успешно создан.
